# LOAD THE COEFFICIENTS, SUMMARY AND PROBABILITIES

In [2]:
import pandas as pd
import numpy as np

In [3]:
final_model_coefficients=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\final_model_coefficients.csv")
model_summary=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\model_summary.csv")
test_pd=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\test_pd.csv")
train_pd=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\train_pd.csv")

In [35]:
x_train_final=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_train_final.csv")

In [94]:
x_test_final=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_test_final.csv")

# FIX THE SCORE DESIGN PARAMETERS
1) BASE SCORE: 600
2) PDO: 20
3) BASE ODDS: 50

In [6]:
base_odds=50
base_score=600
pdo=20

In [4]:
# CALCULATE THE FACTOR THAT NEEDS TO BE MULTIPLIED BY COEFFICIENTS AND WOE 
# FACTOR= PDO/ln(2)
factor=(20/np.log(2))
factor

np.float64(28.85390081777927)

In [7]:
# CALCUATE OFFSET
# OFFSET= BASE SCORE - FACTOR * ln(base odds)
offset=base_score- (factor*np.log(base_odds))
offset

np.float64(487.1228762045055)

# COMPUTE BASE POINTS (INTERCEPT CONTRIBUTION)

In [9]:
final_model_coefficients

,Unnamed: 0,0
0,const,-2.435404
1,EXT_SOURCE_3,-0.847634
2,EXT_SOURCE_2,-0.816845
3,EXT_SOURCE_1,-0.530618
4,DAYS_EMPLOYED,-0.342289
5,AMT_GOODS_PRICE,-0.650568
6,OCCUPATION_TYPE,-0.323617
7,ORGANIZATION_TYPE,-0.415435
8,NAME_INCOME_TYPE,-0.089649
9,NAME_EDUCATION_TYPE,-0.623591


In [11]:
intercept=final_model_coefficients.iloc[0,1]
intercept

np.float64(-2.435403944405365)

In [12]:
base_points=offset-(factor*intercept)
base_points

np.float64(557.3937800676063)

# Calculate WOE Points for each bin 

In [13]:
#import the WOE tables
import pickle
with open("models/model_artifacts.pkl", "rb") as f:
    artifacts = pickle.load(f)

selected_vars = artifacts["selected_vars"]
woe_tables = artifacts["woe_tables"]
bin_edges_dict = artifacts["bin_edges_dict"]

In [14]:
woe_tables['EXT_SOURCE_2']

TARGET,0,1,%good,%bad,WOE,IV_COMPONENT
EXT_SOURCE_2_BIN,,,,,,
"(-0.0009999183, 0.217]",20096,4514,0.088862,0.227291,-0.939145,0.130005
"(0.217, 0.341]",21612,2980,0.095566,0.150050,-0.451157,0.024581
"(0.341, 0.441]",22156,2446,0.097971,0.123162,-0.228828,0.005764
"(0.441, 0.513]",22530,2072,0.099625,0.104330,-0.046149,0.000217
"(0.513, 0.566]",22997,1865,0.101690,0.093907,0.079621,0.000620
"(0.566, 0.608]",22741,1596,0.100558,0.080363,0.224187,0.004528
"(0.608, 0.646]",23156,1444,0.102393,0.072709,0.342355,0.010163
"(0.646, 0.682]",23417,1187,0.103547,0.059768,0.549551,0.024059
"(0.682, 0.722]",23565,1033,0.104202,0.052014,0.694813,0.036261


In [18]:
final_model_coefficients.rename(columns={'Unnamed: 0':'Variables','0':"Coeff"},inplace=True)

In [30]:
coef_var=final_model_coefficients[final_model_coefficients["Variables"]=='EXT_SOURCE_2']["Coeff"].values[0]
type(coef_var)

numpy.float64

In [31]:
woe_tables['EXT_SOURCE_2']['BIN_PTS']=-1*coef_var*factor*woe_tables['EXT_SOURCE_2']['WOE']
woe_tables['EXT_SOURCE_2']

TARGET,0,1,%good,%bad,WOE,IV_COMPONENT,BIN_PTS
EXT_SOURCE_2_BIN,,,,,,,
"(-0.0009999183, 0.217]",20096,4514,0.088862,0.227291,-0.939145,0.130005,-22.134850
"(0.217, 0.341]",21612,2980,0.095566,0.150050,-0.451157,0.024581,-10.633379
"(0.341, 0.441]",22156,2446,0.097971,0.123162,-0.228828,0.005764,-5.393275
"(0.441, 0.513]",22530,2072,0.099625,0.104330,-0.046149,0.000217,-1.087683
"(0.513, 0.566]",22997,1865,0.101690,0.093907,0.079621,0.000620,1.876593
"(0.566, 0.608]",22741,1596,0.100558,0.080363,0.224187,0.004528,5.283897
"(0.608, 0.646]",23156,1444,0.102393,0.072709,0.342355,0.010163,8.069015
"(0.646, 0.682]",23417,1187,0.103547,0.059768,0.549551,0.024059,12.952456
"(0.682, 0.722]",23565,1033,0.104202,0.052014,0.694813,0.036261,16.376163


In [32]:
woe_bin_pts={}

In [40]:
def calculate_bin_pts(variable):
    df=woe_tables[variable]
    coeff_var=final_model_coefficients.loc[(final_model_coefficients["Variables"]==variable),"Coeff"].values[0]
    df["BIN_PTS"]=-factor*coeff_var*df['WOE']
    woe_bin_pts[variable]=df

In [41]:
calculate_bin_pts('EXT_SOURCE_3')
woe_bin_pts['EXT_SOURCE_3']

TARGET,0,1,%good,%bad,WOE,IV_COMPONENT,BIN_PTS
EXT_SOURCE_3_BIN,,,,,,,
"(-0.000473, 0.257]",19984,4629,0.088367,0.233082,-0.969891,0.140357,-23.721146
"(0.257, 0.371]",22023,2798,0.097383,0.140886,-0.369300,0.016066,-9.032158
"(0.371, 0.46]",22676,2126,0.100271,0.107049,-0.065417,0.000443,-1.599943
"(0.46, 0.537]",67412,6145,0.298088,0.309416,-0.037298,0.000423,-0.912210
"(0.537, 0.606]",23561,1321,0.104184,0.066516,0.448722,0.016903,10.974631
"(0.606, 0.669]",23310,1111,0.103074,0.055942,0.611140,0.028805,14.946980
"(0.669, 0.735]",23507,931,0.103945,0.046878,0.796312,0.045443,19.475839
"(0.735, 0.896]",23675,799,0.104688,0.040232,0.956332,0.061642,23.389528


In [42]:
for col in x_train_final.columns.tolist():
    calculate_bin_pts(col)

# Build the final scorecard 

In [43]:
x_train_final.head()

,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,DAYS_EMPLOYED,AMT_GOODS_PRICE,OCCUPATION_TYPE,ORGANIZATION_TYPE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE
0,-0.037298,-0.228828,-0.049847,0.336116,0.045561,0.235234,0.432406,0.434849,-0.114665
1,0.956332,0.079621,-0.049847,-0.367960,0.220342,-0.303906,-0.434254,-0.191389,-0.114665
2,0.796312,0.549551,0.483362,0.336116,0.231366,0.235234,0.432406,0.434849,-0.114665
3,-0.969891,-0.939145,-0.049847,-0.192015,-0.557737,0.311292,-0.150859,-0.191389,-0.114665
4,-0.037298,-0.046149,0.987788,-0.033212,-0.044244,0.235234,-0.150859,-0.191389,-0.114665


In [46]:
test=x_train_final.iloc[[0],:]
test

,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,DAYS_EMPLOYED,AMT_GOODS_PRICE,OCCUPATION_TYPE,ORGANIZATION_TYPE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE
0,-0.037298,-0.228828,-0.049847,0.336116,0.045561,0.235234,0.432406,0.434849,-0.114665


In [57]:
test["EXT_SOURCE_3"].values[0]

np.float64(-0.0372976929841268)

In [86]:
abs(woe_bin_pts['EXT_SOURCE_3']['WOE']-test["EXT_SOURCE_3"].values[0])<1e-6


EXT_SOURCE_3_BIN
(-0.000473, 0.257]    False
(0.257, 0.371]        False
(0.371, 0.46]         False
(0.46, 0.537]          True
(0.537, 0.606]        False
(0.606, 0.669]        False
(0.669, 0.735]        False
(0.735, 0.896]        False
Name: WOE, dtype: bool

In [87]:
score_scorecard = base_points

for col in x_train_final.columns:
    beta = final_model_coefficients.loc[
        final_model_coefficients["Variables"] == col,
        "Coeff"
    ].values[0]
    
    woe_val = test[col]
    
    score_scorecard -= factor * beta * woe_val

print("Score (Scorecard method):", score_scorecard)

Score (Scorecard method): 0    560.941367
Name: EXT_SOURCE_3, dtype: float64


In [88]:
# compute logit manually
logit = intercept

for col in x_train_final.columns:
    beta = final_model_coefficients.loc[
        final_model_coefficients["Variables"] == col,
        "Coeff"
    ].values[0]
    
    logit += beta * test[col]

score_logistic = offset - factor * logit

print("Score (Logistic method):", score_logistic)

Score (Logistic method): 0    560.941367
Name: EXT_SOURCE_3, dtype: float64


# CREATE A FUNCTION TO AUTOMATE SCORES FOR ALL VARIABLES

In [89]:
x_train_final.head()

,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,DAYS_EMPLOYED,AMT_GOODS_PRICE,OCCUPATION_TYPE,ORGANIZATION_TYPE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE
0,-0.037298,-0.228828,-0.049847,0.336116,0.045561,0.235234,0.432406,0.434849,-0.114665
1,0.956332,0.079621,-0.049847,-0.367960,0.220342,-0.303906,-0.434254,-0.191389,-0.114665
2,0.796312,0.549551,0.483362,0.336116,0.231366,0.235234,0.432406,0.434849,-0.114665
3,-0.969891,-0.939145,-0.049847,-0.192015,-0.557737,0.311292,-0.150859,-0.191389,-0.114665
4,-0.037298,-0.046149,0.987788,-0.033212,-0.044244,0.235234,-0.150859,-0.191389,-0.114665


In [90]:
def calculate_score(model_coefficients,base_points,dataframe,factor):
    score=base_points
    for col in dataframe.columns.tolist():
        beta=model_coefficients.loc[model_coefficients["Variables"]==col,"Coeff"].values[0]
        score-=factor*beta*dataframe[col]
    dataframe["SCORE"]=score
    return dataframe

In [91]:
x_train_scores=calculate_score(final_model_coefficients,base_points,x_train_final,factor)
x_train_scores.head()

,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,DAYS_EMPLOYED,AMT_GOODS_PRICE,OCCUPATION_TYPE,ORGANIZATION_TYPE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,SCORE
0,-0.037298,-0.228828,-0.049847,0.336116,0.045561,0.235234,0.432406,0.434849,-0.114665,560.941367
1,0.956332,0.079621,-0.049847,-0.367960,0.220342,-0.303906,-0.434254,-0.191389,-0.114665,571.797383
2,0.796312,0.549551,0.483362,0.336116,0.231366,0.235234,0.432406,0.434849,-0.114665,611.326604
3,-0.969891,-0.939145,-0.049847,-0.192015,-0.557737,0.311292,-0.150859,-0.191389,-0.114665,496.948835
4,-0.037298,-0.046149,0.987788,-0.033212,-0.044244,0.235234,-0.150859,-0.191389,-0.114665,567.188715


In [95]:
x_test_scores=calculate_score(final_model_coefficients,base_points,x_test_final,factor)
x_test_scores.head()

,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,DAYS_EMPLOYED,AMT_GOODS_PRICE,OCCUPATION_TYPE,ORGANIZATION_TYPE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,SCORE
0,-0.369300,0.342355,-0.011893,-0.374438,-0.044244,0.281957,0.331053,0.366365,0.454906,567.453859
1,0.796312,0.079621,-0.011893,0.571947,-0.116798,-0.285927,-0.028921,-0.191389,-0.114665,576.445644
2,-0.065417,0.224187,-0.049847,0.301072,-0.044244,-0.303906,-0.150859,-0.191389,-0.114665,555.253204
3,0.956332,1.064822,-0.049847,0.571947,-0.557737,0.311292,0.062787,0.088176,-0.114665,602.120594
4,-0.037298,0.079621,-0.611732,-0.192015,-0.044244,-0.303906,-0.434254,-0.191389,-0.114665,535.663997


In [96]:
x_train_scores.to_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_train_scores.csv",index=False)
x_test_scores.to_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_test_scores.csv",index=False)